In [11]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

# Załaduj dane i przygotuj szereg
df = pd.read_excel("turystyka_czasowe_2.xlsx")
df["Data"] = pd.to_datetime(df["Data"])
df.set_index("Data", inplace=True)
y = df["Wartosc"].dropna().asfreq("MS")
y_log = np.log(y).replace([np.inf, -np.inf], np.nan).dropna()

# STL – odsezonowanie
from statsmodels.tsa.seasonal import STL
stl = STL(y_log, period=12, robust=True).fit()
y_log_sa = y_log - stl.seasonal

# Zakres modeli (p,1,q)
p_max = 5
q_max = 4

results = []

for p in range(p_max + 1):
    for q in range(q_max + 1):
        try:
            model = ARIMA(y_log_sa, order=(p, 1, q)).fit()
            ljung_pval = acorr_ljungbox(model.resid, lags=[10], return_df=True)["lb_pvalue"].values[0]

            results.append({
                "model": f"ARIMA({p},{1},{q})",
                "AIC": model.aic,
                "BIC": model.bic,
                "HQC": model.hqic,
                "LB_p(10)": ljung_pval,
                "logLik": model.llf
            })

        except Exception as e:
            print(f"Błąd dla ARIMA({p},1,{q}): {e}")

# Zapisz do DataFrame i posortuj
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("AIC").reset_index(drop=True)

# Wyświetl tabelę wyników
print(df_results.to_string(index=False))



/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: overflow encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: invalid value encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)

       model       AIC       BIC       HQC  LB_p(10)     logLik
ARIMA(3,1,4) 21.638360 46.724747 31.818125       1.0  -2.819180
ARIMA(4,1,3) 22.078446 47.164833 32.258211       1.0  -3.039223
ARIMA(3,1,3) 23.213320 45.163909 32.120615       1.0  -4.606660
ARIMA(2,1,3) 23.220487 42.035278 30.855311       1.0  -5.610243
ARIMA(5,1,3) 23.513464 51.735650 34.965700       1.0  -2.756732
ARIMA(3,1,2) 30.101744 48.916535 37.736568       1.0  -9.050872
ARIMA(4,1,2) 31.257690 53.208279 40.164984       1.0  -8.628845
ARIMA(1,1,4) 31.286402 50.101193 38.921226       1.0  -9.643201
ARIMA(2,1,1) 31.802674 44.345867 36.892556       1.0 -11.901337
ARIMA(2,1,4) 32.641984 54.592573 41.549278       1.0  -9.320992
ARIMA(5,1,2) 32.884962 57.971350 43.064727       1.0  -8.442481
ARIMA(5,1,4) 33.005448 64.363433 45.730155       1.0  -6.502724
ARIMA(0,1,4) 33.085384 48.764376 39.447737       1.0 -11.542692
ARIMA(1,1,2) 33.587916 46.131109 38.677798       1.0 -12.793958
ARIMA(3,1,1) 33.724718 49.403710 40.0870

/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: overflow encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: invalid value encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters f

In [13]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox, acorr_breusch_godfrey

# Załaduj dane i przygotuj szereg
df = pd.read_excel("bezrobocie_czasowe.xlsx")
df["Data"] = pd.to_datetime(df["Data"])
df.set_index("Data", inplace=True)
y = df["Wartosc"].dropna().asfreq("MS")
y_log = np.log(y).replace([np.inf, -np.inf], np.nan).dropna()

# STL – odsezonowanie
from statsmodels.tsa.seasonal import STL
stl = STL(y_log, period=12, robust=True).fit()
y_log_sa = y_log - stl.seasonal

# Zakres modeli (p,1,q)
p_max = 5
q_max = 4

results = []

for p in range(p_max + 1):
    for q in range(q_max + 1):
        try:
            model = ARIMA(y_log_sa, order=(p, 1, q)).fit()
            resid = model.resid.dropna()

            # test Ljunga–Box (opcjonalnie trzymasz go nadal)
            lb_p10 = acorr_ljungbox(resid, lags=[10], return_df=True)["lb_pvalue"].iloc[0]

            # ** TYLKO test Breuscha–Godfreya dla nlags=10 **
            bg_chi2, bg_pval, _, _ = acorr_breusch_godfrey(model, nlags=10)

            results.append({
                "model": f"ARIMA({p},1,{q})",
                "AIC": model.aic,
                "BIC": model.bic,
                "HQC": model.hqic,
                "LB_p(10)": lb_p10,
                "BG_p(10)": bg_pval,
                "logLik": model.llf
            })

        except Exception as e:
            print(f"Błąd dla ARIMA({p},1,{q}): {e}")

# Zapisz do DataFrame i posortuj
df_results = pd.DataFrame(results).sort_values("AIC").reset_index(drop=True)
print(df_results.to_string(index=False))


/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: overflow encountered in matmul
  return _core_matmul(x1, x2)
/Users/jakubwojna/Documents/ASC/python/venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: inval

       model          AIC         BIC         HQC  LB_p(10)  BG_p(10)     logLik
ARIMA(5,1,3) -1006.689029 -978.414057 -995.216249       1.0       1.0 512.344515
ARIMA(5,1,4) -1003.469254 -972.052618 -990.721720       1.0       1.0 511.734627
ARIMA(5,1,1)  -998.754794 -976.763149 -989.831521       1.0       1.0 506.377397
ARIMA(5,1,0)  -994.039120 -975.189139 -986.390600       1.0       1.0 503.019560
ARIMA(5,1,2)  -993.817336 -968.684028 -983.619309       1.0       1.0 504.908668
ARIMA(0,1,4)  -987.090126 -971.381808 -980.716359       1.0       1.0 498.545063
ARIMA(1,1,3)  -986.600920 -970.892602 -980.227153       1.0       1.0 498.300460
ARIMA(3,1,3)  -984.820871 -962.829226 -975.897598       1.0       1.0 499.410436
ARIMA(1,1,4)  -984.158755 -965.308773 -976.510234       1.0       1.0 498.079377
ARIMA(4,1,3)  -983.183979 -958.050671 -972.985952       1.0       1.0 499.591990
ARIMA(4,1,4)  -980.154304 -951.879332 -968.681524       1.0       1.0 499.077152
ARIMA(0,1,3)  -976.303945 -9